In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q gem-suite

*Zickduur: Öm un bei 3 Minute op enem Heron r2 Prozessor (OPJEPASS: Dat es bloß e Schätzung. Ding Laufzick künnt angers sin.)*

## Hintergrund

Hee weed jezeich, wi mer e Nishimori Phasenübergang op enem IBM&reg; Quanteprozessor usföhre kann. Dat Experiment wohr ursprünglich beschrevve en [*Realizing the Nishimori transition across the error threshold for constant-depth quantum circuits*](https://arxiv.org/abs/2309.02863).

D'r Nishimori Phasenübergang bezich sich op d'r Übergang zwesche kooze un langstrecke jeordnete Phase em Zufalls-Bindungs-Ising-Modell. Op enem Quantecomputer manifesteet sich de langstrecke jeordnete Phase als e Zostand, wo de Qubits üvver dat janze Jerät verschränk sin. Dä hochgradig verschränkte Zostand weed met dem *generation of entanglement by measurement* (GEM) Protokoll eruzeuch. Durchs Bruche vun Messunge metzte em Schaltkreis kann dat GEM-Protokoll Qubits üvver dat janze Jerät verschränke, met Schaltkreis vun bloß konstanter Deepde. Hee bruche mer de Implementation vum GEM-Protokoll us dem [GEM Suite](https://github.com/qiskit-community/gem-suite) Software-Paket.

## Vorrausetzunge

Bevör de met däm Tutorial aanfängks, sorg doför, dat de dat hee installeert häs:

- Qiskit SDK v1.0 odder neuer, met [Visualisierung](https://docs.quantum.ibm.com/api/qiskit/visualization) Ongerstötzung
- Qiskit Runtime v0.22 odder neuer ( `pip install qiskit-ibm-runtime` )
- GEM Suite ( `pip install gem-suite` )

## Vorbereitung

In [2]:
import matplotlib.pyplot as plt

from collections import defaultdict

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager

from gem_suite import PlaquetteLattice
from gem_suite.experiments import GemExperiment

## Schrett 1: Klassische Eingabe op e Quanteproblem afbelde
Dat GEM-Protokoll arbed op enem Quanteprozessor met Qubit-Konnektivität, die durch e Jitter beschrevve weed. De hüdige IBM Quanteprozessore bruche dat [Heavy-Hex-Jitter](https://www.ibm.com/quantum/blog/heavy-hex-lattice). De Qubits vum Prozessor wore en *Plaquettes* jeruppeert, jenaach en wellere Eenheitzell vum Jitter se ligge. Weil e Qubit en mih wi einem Eenheitzell optredd kann, sin de Plaquettes nit disjunkt. Op dem Heavy-Hex-Jitter hät e Plaquette 12 Qubits. De Plaquettes selvs belde och e Jitter, wo zwei Plaquettes verbunge sin, wann se irgendwelche Qubits deile. Op dem Heavy-Hex-Jitter deile benohbote Plaquettes 3 Qubits.

Em GEM Suite Software-Paket es de Jrundklasse för d'r Implementation vum GEM-Protokoll `PlaquetteLattice`, wat dat Jitter vun de Plaquettes repräsenteert (wat angers es wi dat Heavy-Hex-Jitter). E `PlaquetteLattice` kann us enem Qubit Coupling Map initialiseet weede. Momentan weede bloß Heavy-Hex Coupling Maps ongerstötzt.

Dä nächste Code-Zell initialiseet e Plaquette-Jitter us dem Coupling Map vun enem IBM Quanteprozessor. Dat Plaquette-Jitter ömfasst nit emmer de janze Hardware. Zom Beispill hät `ibm_torino` 133 Qubits jesamt, ävver dat jrüßte Plaquette-Jitter, wat op dat Jerät pass, bruch bloß 125 dovun un ömfasst jesamt 18 Plaquettes. Et Ähnliches kann mer och bei IBM Quantum&reg; Jeräte met angren Qubit-Aanzahle beovachte.

In [ ]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_API_KEYN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
plaquette_lattice = PlaquetteLattice.from_coupling_map(backend.coupling_map)

print(f"Number of qubits in backend: {backend.num_qubits}")
print(
    f"Number of qubits in plaquette lattice: {len(list(plaquette_lattice.qubits()))}"
)
print(f"Number of plaquettes: {len(list(plaquette_lattice.plaquettes()))}")

Number of qubits in backend: 133
Number of qubits in plaquette lattice: 125
Number of plaquettes: 18


You can visualize the plaquette lattice by generating a diagram of its graph representation. In the diagram, the plaquettes are represented by labeled hexagons, and two plaquettes are connected by an edge if they share qubits.

In [7]:
plaquette_lattice.draw_plaquettes()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif" alt="Output of the previous code cell" />

Do kanns dat Plaquette-Jitter visualiseere, en däm do e Diajramm vun singem Graph-Repräsentation eruzeuchs. Em Diajramm weede de Plaquettes als beschriftete Sechsecke dorjestellt, un zwei Plaquettes sin durchs e Kante verbunge, wann se Qubits deile.

In [8]:
# Get a list of the plaquettes
plaquettes = list(plaquette_lattice.plaquettes())
# Display information about plaquette 0
plaquettes[0]

PyPlaquette(index=0, qubits=[0, 1, 2, 3, 4, 15, 16, 19, 20, 21, 22, 23], neighbors=[3, 1])

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif)

Do kanns Informatione üvver einzelne Plaquettes ophale, wi zom Beispill de Qubits, die se enthalde, met der `plaquettes` Methode.

In [9]:
plaquette_lattice.draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif" alt="Output of the previous code cell" />

In addition to the qubit labels and the edges indicating which qubits are connected, the diagram contains three additional pieces of information that are relevant to the GEM protocol:
- Each qubit is either shaded (gray) or unshaded. The shaded qubits are "site" qubits that represent the sites of the Ising model, and the unshaded qubits are "bond" qubits used to mediate interactions between the site qubits.
- Each site qubit is labeled either (A) or (B), indicating one of two roles a site qubit can play in the GEM protocol (the roles are explained later).
- Each edge is colored using one of six colors, thus partitioning the edges into six groups. This partitioning determines how two-qubit gates can be parallelized, as well as different scheduling patterns that are likely to incur different amounts of error on a noisy quantum processor. Because edges in a group are disjoint, a layer of two-qubit gates can be applied on those edges simultaneously. In fact, it is possible to partition the six colors into three groups of two colors such that the union of each group of two colors is still disjoint. Therefore, only three layers of two-qubit gates are needed to activate every edge. There are 12 ways to so partition the six colors, and each such partition yields a different 3-layer gate schedule.

Now that you have created a plaquette lattice, the next step is to initialize a `GemExperiment` object, passing both the plaquette lattice and the backend that you intend to run the experiment on. The `GemExperiment` class manages the actual implementation of the GEM protocol, including generating circuits, submitting jobs, and analyzing the data. The following code cell initializes the experiment class while restricting the plaquette lattice to only two of the plaquettes (21 qubits), reducing the size of the experiment to ensure that the noise in the hardware doesn't overwhelm the signal.

In [16]:
gem_exp = GemExperiment(plaquette_lattice.filter([9, 12]), backend=backend)

# visualize the plaquette lattice after filtering
plaquette_lattice.filter([9, 12]).draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif" alt="Output of the previous code cell" />

Do kanns och e Diajramm vun de zu Jrund liegende Qubits eruzeuch, die dat Plaquette-Jitter belde.

In [12]:
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

Total number of circuits: 252


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif)

Zosätzlich zo de Qubit-Bezeichnunge un de Kante, die aanzeije, wellche Qubits verbunge sin, enthällt dat Diajramm drei widdere Informatione, die för dat GEM-Protokoll relevant sin:
- Jedes Qubit es entweder schatteert (jrau) odder nit schatteert. De schatteerte Qubits sin "Site"-Qubits, die de Plätz vum Ising-Modell repräsenteere, un de nit schatteerte Qubits sin "Bond"-Qubits, die dozoh brucht weede, öm Interaktione zwesche de Site-Qubits ze vermittele.
- Jedes Site-Qubit es entweder (A) odder (B) beschriftet, wat ein vun zwei Rolle aanzeich, die e Site-Qubit em GEM-Protokoll spille kann (de Rolle weede spööder erklärt).
- Jede Kant es met einer vun sechs Farve jefärv, wat de Kante en sechs Jroppe opdeilt. Dat Opdeilung bestemmp, wi Zwei-Qubit-Jatter parallelliseert weede künne, esu wi verschiddene Zeitplane, die wohrscheinlich verschiddene Feelerhöhe op enem rauschende Quanteprozessor verursache. Weil de Kante en enem Jropp disjunkt sin, kann e Schicht vun Zwei-Qubit-Jatter op dänne Kante jlichziggisch aanjebraat weede. Tatsächlich kann mer de sechs Farve en drei Jroppe vun zwo Farve opdeilung, su dat de Vereenigung vun jedere Jropp vun zwei Farve noch emmer disjunkt es. Doröm weede bloß drei Schichte vun Zwei-Qubit-Jatter jebrucht, öm jede Kant ze aktiviere. Et jitt 12 Wäje, de sechs Farve su opdezeilung, un jede su en Opdeilung levvet e angere 3-Schichte-Jatter-Zeitplaan.

Nohm dat do e Plaquette-Jitter eruzeuch häs, es d'r nächste Schrett, e `GemExperiment` Objekt ze initialisiere, woh mer beeds dat Plaquette-Jitter un dat Backend reenjitt, op dem mer dat Experiment durchföhre wells. De `GemExperiment` Klasse verwalltet de eigentliche Implementation vum GEM-Protokoll, wat och Schaltkreis eruzeuch, Jobs avscheck un Daate analyseet. Dä nächste Code-Zell initialiseet de Experiment-Klasse un beschränk dat Plaquette-Jitter op bloß zwei vun de Plaquettes (21 Qubits), öm de Jrüß vum Experiment ze reduziere un secher ze stelle, dat dat Rausche en d'r Hardware nit dat Signal üvverwältich.

In [13]:
# Restrict experiment to the first scheduling pattern
gem_exp.set_experiment_options(schedule_idx=0)

# There are less circuits now
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

# Print the RZZ angles swept over
print(f"RZZ angles:\n{gem_exp.parameters()}")

Total number of circuits: 21
RZZ angles:
[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651 1.57079633]


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif)

E GEM-Protokoll-Schaltkreis weed met dänne Schrette jebaut:
1. Eruzeuch dat all-$|+\rangle$ Zostand, en däm mer e Hadamard-Jatter op jedes Qubit aanlääje.
2. Lääje e $R_{ZZ}$ Jatter zwesche jedem Paar vun verbungene Qubits aan. Dat kann mer met 3 Schichte vun Jatter erreeche. Jedes $R_{ZZ}$ Jatter wirk op e Site-Qubit un e Bond-Qubit. Wann dat Site-Qubit (B) beschriftet es, dann es d'r Winkel fess op $\frac{\pi}{2}$ jesatz. Wann dat Site-Qubit (A) beschriftet es, dann darf d'r Winkel variiere, wat angere Schaltkreis eruzeuch. Standardmäßisch es d'r Bereich vun Winkele op 21 jlichmäßich verdeilte Pünk zwesche $0$ un $\frac{\pi}{2}$, inklusive, jesatz.
3. Mess jedes Bond-Qubit en d'r Pauli $X$ Basis. Weil Qubits en d'r Pauli $Z$ Basis jemesse weede, kann mer dat maache, en däm mer e Hadamard-Jatter vör d'r Messung aanlääje.

Op de mööch, dat de Papier, die em Inleiding vun däm Tutorial ziteert weed, e angere Konvention för d'r $R_{ZZ}$ Winkel bruch, die sich durch enem Faktor vun 2 vun d'r Konvention en däm Tutorial ongerscheidt.

Em Schrett 3 weede bloß de Bond-Qubits jemesse. Öm ze verstohn, en welchem Zostand de Site-Qubits blieve, es et hilfreich, d'r Fall ze betrachte, dat d'r $R_{ZZ}$ Winkel, dä em Schrett 2 op Site-Qubits (A) aanjelaat weed, jlich $\frac{\pi}{2}$ es. En däm Fall blieve de Site-Qubits en enem hochjradig verschränkte Zostand, ähnlich wi beim GHZ-Zostand,

$$
\lvert \text{GHZ} \rangle = \lvert 00 \cdots 00 \rangle + \lvert 11 \cdots 11 \rangle.
$$

Wäje d'r Zufälligkeit en de Messungserjebnisse künnt d'r eigentliche Zostand vun de Site-Qubits e angere Zostand met langstrecker Ordnung sin, zom Beispill, $\lvert 00110 \rangle + \lvert 11001 \rangle$. Ävver d'r GHZ-Zostand kann widder herojevonge weede, en däm mer e Dekodeerungs-Operation baseerend op de Messungserjebnisse aanlääje. Wann d'r $R_{ZZ}$ Winkel vun $\frac{\pi}{2}$ eronjerönnt weed, kann de langstreck Ordnung noch emmer widder herojevonge weede bes zo enem kritische Winkel, dä en Afwäseheit vun Rausche ömmerhin öm $0.3 \pi$ es. Onger däm Winkel zeich d'r Zostand, dä eruzeuch weed, nit mih langstreck Verschränkung. Dä Übergang zwesche d'r Aanwäseheit un Afwäseheit vun langstrecker Ordnung es d'r Nishimori Phasenübergang.

En d'r Beschrievung hee bovve wore de Site-Qubits nit jemesse, un de Dekodeerungs-Operation kann durchjeföhrt weede, en däm mer Quantejatter aanlääje. Em Experiment, wi et em GEM Suite implementeet es, wat däm Tutorial fölg, weede de Site-Qubits tatsächlich jemesse, un de Dekodeerungs-Operation weed en enem klassische Nohbearbeitung-Schrett aanjelaat.

En d'r Beschrievung hee bovve kann de Dekodeerungs-Operation durchjeföhrt weede, en däm mer Quantejatter op de Site-Qubits aanlääje, öm d'r Quantezostand widder herozvekrigge. Ävver, wann dat Ziel es, d'r Zostand soffott ze messe, zom Beispill för Charakteriseerungs-Zweck, dann weede de Site-Qubits zosamme met de Bond-Qubits jemesse, un de Dekodeerungs-Operation kann en enem klassische Nohbearbeitung-Schrett aanjelaat weede. Esu es dat Experiment em GEM Suite implementeet, wat däm Tutorial fölg.

Zosätzlich dodozoh, dat et vum $R_{ZZ}$ Winkel em Schrett 2 avhängich es, dä standardmäßisch üvver 21 Wäät jeht, hängk d'r GEM-Protokoll-Schaltkreis och vun dem Zeitplaan-Muster av, dat brucht weed, öm de 3 Schichte vun $R_{ZZ}$ Jattere ze implementiere. Wi vörheer besprooche jitt et 12 su Zeitplaan-Muster. Doröm es de jesamte Aanzahl vun Schaltkreis em Experiment $21 \times 12 = 252$.

De Schaltkreis vum Experiment künne met d'r `circuits` Methode vun d'r `GemExperiment` Klasse eruzeuch weede.

In [14]:
# Get the circuit at index 5
circuit = circuits[5]
# Remove the final measurements to ease visualization
circuit.remove_final_measurements()
# Draw the circuit
circuit.draw("mpl", fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Transpiling quantum circuits for execution on hardware typically involves a [number of stages](/docs/guides/transpiler-stages). Typically, the stages that incur the most computational overhead are choosing the qubit layout, routing the two-qubit gates to conform to the qubit connectivity of the hardware, and optimizing the circuit to minimize its gate count and depth. In the GEM protocol, the layout and routing stages are unnecessary because the hardware connectivity is already incorporated into the design of the protocol. The circuits already have a qubit layout, and the two-qubit gates are already mapped onto native connections. Furthermore, in order to preserve the structure of the circuit as the $R_{ZZ}$ angle is varied, only very basic circuit optimization should be performed.

The `GemExperiment` class transparently transpiles circuits when executing the experiment. The layout and routing stages are already overridden by default to do nothing, and circuit optimization is performed at a level that only optimizes single-qubit gates. However, you can override or pass additional options using the `set_transpile_options` method. For the sake of visualization, the following code cell manually transpiles the circuit displayed previously, and draws the transpiled circuit.

In [15]:
# Demonstrate setting transpile options
gem_exp.set_transpile_options(
    optimization_level=1  # This is the default optimization level
)
pass_manager = generate_preset_pass_manager(
    backend=backend,
    initial_layout=list(gem_exp.physical_qubits),
    **dict(gem_exp.transpile_options),
)
transpiled = pass_manager.run(circuit)
transpiled.draw("mpl", idle_wires=False, fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif" alt="Output of the previous code cell" />

För de Zweck vun däm Tutorial es et jenoch, bloß e einzelnet Zeitplaan-Muster ze betrachte. Dä nächste Code-Zell beschränk dat Experiment op dat eeschte Zeitplaan-Muster. Domet hät dat Experiment bloß 21 Schaltkreis, eine för jede $R_{ZZ}$ Winkel, üvver dä jejange weed.

In [10]:
exp_data = gem_exp.run(shots=10_000)

To wait for the results, call the `block_for_results` method of the `ExperimentData` object. This call will cause the interpreter to hang until the jobs are finished.

In [11]:
exp_data.block_for_results()

ExperimentData(GemExperiment, d0d5880a-34c1-4aab-a7b6-c4f58516bc03, job_ids=['cwg12ptmptp00082khhg'], metadata=<5 items>, figure_names=['two_point_correlation.svg', 'normalized_variance.svg', 'plaquette_ops.svg', 'bond_ops.svg'])

Dä nächste Code-Zell zeich e Diajramm vum Schaltkreis bei Index 5. Öm de Jrüß vum Diajramm ze reduziere, weede de Messungs-Jatter am Eng vum Schaltkreis fottjenoohme.

In [ ]:
def magnetization_distribution(
    counts_dict: dict[str, int],
) -> dict[str, float]:
    """Compute magnetization distribution from counts dictionary."""
    # Construct dictionary from magnetization to count
    mag_dist = defaultdict(float)
    for bitstring, count in counts_dict.items():
        mag = bitstring.count("0") - bitstring.count("1")
        mag_dist[mag] += count
    # Normalize
    shots = sum(counts_dict.values())
    for mag in mag_dist:
        mag_dist[mag] /= shots
    return mag_dist


# Get counts dictionaries with and without decoding
data = exp_data.data()
# Get the last data point, which is at the angle for the GHZ state
raw_counts = data[-1]["counts"]
# Without decoding
site_indices = [
    i for i, q in enumerate(gem_exp.plaquettes.qubits()) if q.role == "Site"
]
site_raw_counts = defaultdict(int)
for key, val in raw_counts.items():
    site_str = "".join(key[-1 - i] for i in site_indices)
    site_raw_counts[site_str] += val
# With decoding
_, site_decoded_counts = gem_exp.plaquettes.decode_outcomes(
    raw_counts, return_counts=True
)

# Compute magnetization distribution
raw_magnetization = magnetization_distribution(site_raw_counts)
decoded_magnetization = magnetization_distribution(site_decoded_counts)

# Plot
plt.bar(*zip(*raw_magnetization.items()), label="raw")
plt.bar(*zip(*decoded_magnetization.items()), label="decoded", width=0.3)
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Frequency")
plt.title("Magnetization distribution with and without decoding")

Text(0.5, 1.0, 'Magnetization distribution with and without decoding')

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/8ead3582-16df-4616-836c-bdce867ad6b8-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif)

## Schrett 2: Problem för Quante-Hardware-Usföhrung optimiere
Transpileerungs-Quanteschaltkreis för de Usföhrung op Hardware ömfasst typischerwiis [enne janze Reihe vun Etappe](/guides/transpiler-stages). Typischerwiis sin de Etappe, die am mihste rechenintensive sin, de Uswahl vum Qubit-Layout, dat Routing vun de Zwei-Qubit-Jattere, öm sich de Qubit-Konnektivität vun d'r Hardware aanzepasse, un de Optimeerung vum Schaltkreis, öm singem Jatter-Aanzahl un Deepde ze minimiere. Em GEM-Protokoll sin de Layout- un Routing-Etappe ongernüdich, weil de Hardware-Konnektivität schon en dat Design vum Protokoll enjebaad es. De Schaltkreis han schon e Qubit-Layout, un de Zwei-Qubit-Jattere sin schon op native Verbindunge afjebeldt. Dobenevve sull bloß sehr einfache Schaltkreis-Optimeerung durchjeföhrt weede, öm de Struktur vum Schaltkreis ze erhalte, wann d'r $R_{ZZ}$ Winkel varieet weed.

De `GemExperiment` Klasse transpileert Schaltkreis transparent, wann dat Experiment durchjeföhrt weed. De Layout- un Routing-Etappe sin standardmäßisch schon üvverschrevve, öm nix ze dun, un Schaltkreis-Optimeerung weed op enem Nivoh durchjeföhrt, dat bloß Ein-Qubit-Jattere optimeet. Ävver do kanns zusätzliche Optione üvverschrieve odder overjäve, en däm do de `set_transpile_options` Methode bruchs. För de Zweck vun d'r Visualiseerung transpileert dä nächste Code-Zell manuell d'r Schaltkreis, dä vörheer aanjezeich wohr, un zeich d'r transpileet Schaltkreis.

In [13]:
exp_data.figure("two_point_correlation")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/4ecb25c8-e572-49af-a879-9943039db131-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif)

## Schrett 3: Met Qiskit Primitives usföhre
Öm de GEM-Protokoll-Schaltkreis op d'r Hardware uszföhre, roof de `run` Methode vum `GemExperiment` Objekt op. Do kanns de Aanzahl vun Shots aanjäve, die do us jedem Schaltkreis sample wells. De `run` Methode levvet e [ExperimentData](https://qiskit-community.github.io/qiskit-experiments/stubs/qiskit_experiments.framework.ExperimentData.html) Objekt, wat do en enem Variable speichere sulls. Op de mööch, dat de `run` Methode bloß Jobs avscheck, ohne op die ze waade bes se fäärdich sin, esu dat et e nit-blockeerendet Oproof es.

In [14]:
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/2b351d68-3924-445a-94ef-047b16214e8a-0.avif" alt="Output of the previous code cell" />

Öm op de Erjebnisse ze waade, roof de `block_for_results` Methode vum `ExperimentData` Objekt op. Dä Oproof lööt d'r Interpreter hänge, bes de Jobs fäärdich sin.

In [15]:
gem_exp = GemExperiment(
    plaquette_lattice.filter(range(3, 9)), backend=backend
)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/08581c09-a6a5-4a56-9fc4-abf22b063c6a-0.avif" alt="Output of the previous code cell" />

In [16]:
gem_exp = GemExperiment(plaquette_lattice, backend=backend)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/37e9a4cd-6efb-4ade-ad09-8139db9d58e9-0.avif" alt="Output of the previous code cell" />

## Schrett 4: Nohbearbeide un Erjebniss em jewönschte klassische Format zeröckjäve
Bei enem $R_{ZZ}$ Winkel vun $\frac{\pi}{2}$ wööhr d'r dekodeet Zostand d'r GHZ-Zostand en Afwäseheit vun Rausche. De langstreck Ordnung vum GHZ-Zostand kann visualiseet weede, en däm mer de Magnetiseerung vun de jemessene Bitstrings afbeldt. De Magnetiseerung $M$ weed defineert als de Summe vun de Ein-Qubit-Pauli $Z$ Operatore,
$$
M = \sum_{j=1}^N Z_j,
$$
wo $N$ de Aanzahl vun Site-Qubits es. Singem Wäät för e Bitstring es jlich de Ongerscheed zwesche d'r Aanzahl vun Nulls un d'r Aanzahl vun Eins. Wann mer d'r GHZ-Zostand mess, kritt mer d'r all-Null Zostand odder d'r all-Eins Zostand met jlicher Wohrscheinlichkeit, esu dat de Magnetiseerung $+N$ de halve Zick un $-N$ de anger halve Zick wöhr. En d'r Aanwäseheit vun Feller wäje Rausche dääte och anger Wäät optredd, ävver wann dat Rausche nit zo jruß es, dääte de Verdeilung noch emmer Spetze in d'r Nöh vun $+N$ un $-N$ zeije.

För de roh Bitstrings vör d'r Dekodeerung wöhr de Verdeilung vun d'r Magnetiseerung jlichbedütend met dänne vun jlichmäßich zufälliche Bitstrings, en Afwäseheit vun Rausche.

Dä nächste Code-Zell zeich de Magnetiseerung vun de roh Bitstrings un de dekodeerte Bitstrings beim $R_{ZZ}$ Winkel vun $\frac{\pi}{2}$.